In [11]:
!pip install selenium --upgrade
!pip install webdriver_manager --upgrade

In [38]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options


In [46]:
# Ensure chromedriver is set up
ChromeDriverManager().install()

# Initialize the browser
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=chrome_options)

# Set Chrome to automatically download PDF files
prefs = {
    "download.default_directory": "/Users/neuefische/Downloads/test_ESG", 
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": False
}
chrome_options.add_experimental_option('prefs', prefs)

#driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

driver.get("https://www.responsibilityreports.com/")

company_list = ["Apple"]  # List of S&P 500 companies

for company in company_list:
    try:
        # Step 1: Navigate to the website
        driver.get("https://www.responsibilityreports.com/")

        # Step 2: Interact with the search bar
        search_bar = driver.find_element(By.CSS_SELECTOR, "body > div.container > section.banner_section > div > div > div.left_section > form > input[type=text]:nth-child(2)")
        search_bar.send_keys(company)
        search_bar.submit()

        # Step 3: Select the company from search results
        company_link = driver.find_element(By.CSS_SELECTOR, "body > div.container > section.category_section > div.apparel_stores_company_list > ul > li:nth-child(2) > span.companyName > a")
        company_link.click()

        # Step 4: Download the most recent report
        most_recent_link = driver.find_element(By.CSS_SELECTOR, "div.most_recent_content_block a")
        most_recent_link.click()

        # Create a wait object with a timeout of 2 seconds
        wait = WebDriverWait(driver, 2)

        # Close the pop-up if present
        try:
            close_popup = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "body > div.container > div > div > div.close_popup > a")))
            close_popup.click()
        except:
            pass  # If popup is not present, continue to the next step

        # Step 5: Download archived reports
        archived_links = driver.find_elements(By.CSS_SELECTOR, "div.archived_report_content_block a")

        # Click on each archived report link
        for link in archived_links:
            link.click()
            #time.sleep(2)  # wait for 2 seconds to allow the download to initiate

    except Exception as e:
        print(f"Error with company {company}: {e}")
        continue


Error with company Apple: Message: element not interactable
  (Session info: chrome=116.0.5845.187)
Stacktrace:
0   chromedriver                        0x00000001010ae65c chromedriver + 4318812
1   chromedriver                        0x00000001010a6d00 chromedriver + 4287744
2   chromedriver                        0x0000000100cd8644 chromedriver + 296516
3   chromedriver                        0x0000000100d176ac chromedriver + 554668
4   chromedriver                        0x0000000100d0bf90 chromedriver + 507792
5   chromedriver                        0x0000000100d0b858 chromedriver + 505944
6   chromedriver                        0x0000000100d4ed28 chromedriver + 781608
7   chromedriver                        0x0000000100d0a178 chromedriver + 500088
8   chromedriver                        0x0000000100d0afc0 chromedriver + 503744
9   chromedriver                        0x000000010106ec40 chromedriver + 4058176
10  chromedriver                        0x0000000101073160 chromedriver + 4

In [47]:
# exit driver

driver.quit()